Module Description:
-------------------
Compare the performance of current model (GEMMA 2b-it) and LLAMA 3.2-1b-it model (latest light-weight model developed by meta). Introduce the torch compile setup for faster inference.

Ownership:
----------
Project: Leveraging Artificial intelligence for Skills Extraction and Research (LAiSER)

Owner:  

        George Washington University Insitute of Public Policy
        Program on Skills, Credentials and Workforce Policy
        Media and Public Affairs Building
        805 21st Street NW
        Washington, DC 20052
        PSCWP@gwu.edu
        https://gwipp.gwu.edu/program-skills-credentials-workforce-policy-pscwp


License:
--------
Copyright 2024 George Washington University Insitute of Public Policy

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files
(the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify,
merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE
LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR
IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Revision History:
-----------------
Rev No. | Date | Author | Description

---
[1.0.0] | 08/13/2025 | Phanindra Kumar K. | Initial Version

### Download the python package from test pipeline

In [7]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple laiser[gpu]==0.2.43 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 2.0 MB/s eta 0:00:00


### Check package installations and versions

In [8]:
!pip show vllm

Name: vllm
Version: 0.10.0
Summary: A high-throughput and memory-efficient inference and serving engine for LLMs
Home-page: https://github.com/vllm-project/vllm
Author: vLLM Team
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, blake3, cachetools, cbor2, cloudpickle, compressed-tensors, depyf, diskcache, einops, fastapi, filelock, gguf, huggingface-hub, lark, llguidance, lm-format-enforcer, mistral_common, msgspec, ninja, numba, numpy, openai, opencv-python-headless, outlines_core, partial-json-parser, pillow, prometheus-fastapi-instrumentator, prometheus_client, protobuf, psutil, py-cpuinfo, pybase64, pydantic, python-json-logger, pyyaml, pyzmq, ray, regex, requests, scipy, sentencepiece, tiktoken, tokenizers, torch, torchaudio, torchvision, tqdm, transformers, typing_extensions, watchfiles, xformers, xgrammar
Required-by: 


In [9]:
!pip show laiser

Name: laiser
Version: 0.2.43
Summary: Leveraging Artificial Intelligence for Skills Extraction and Research
Home-page: https://github.com/LAiSER-Software/extract-module
Author: 
Author-email: LAiSER Team <PSCWP@gwu.edu>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: faiss-cpu, google-generativeai, ipython, numpy, pandas, psutil, requests, sentence-transformers, skillNer, spacy, torch, tqdm, transformers
Required-by: 


The following codeblock tests
- Skills extraction using legacy pipeline with GPU.
- Skills extraction using refactored pipeline.
- Skills extraction support to multiple LLMs, Gemini API, CPU-only extraction.

In [ ]:
#!/usr/bin/env python3
"""
Test script to verify the refactored skill extractor works with different scenarios
"""

import torch
import pandas as pd
import sys
import os
import argparse

from laiser.skill_extractor import Skill_Extractor
# Import the refactored skill extractor
from laiser.skill_extractor_refactored import SkillExtractorRefactored

# test legacy skill extractor
def test_legacy_skill_extraction():
    """Test the legacy skill extraction functionality"""
    print("Testing legacy skill extraction...")
    # Legacy extraction logic goes here
    use_gpu = True if torch.cuda.is_available() == True else False

    # based on the above arguments, to run this script:
    # python main.py --HF_TOKEN <your_hf_token> --AI_MODEL_ID <your_model_id> --use_gpu True --batch_size 32

    print('\n\nInitializing the Skill Extractor...')
    se = Skill_Extractor(AI_MODEL_ID='TheBloke/Mistral-7B-Instruct-v0.1-AWQ', use_gpu=True)
    print('The Skill Extractor has been initialized successfully!\n')

    # Skill extraction from jobs data
    print('\n\nLoading a sample dataset of 50 jobs...')
    job_sample = pd.read_csv('https://raw.githubusercontent.com/LAiSER-Software/datasets/refs/heads/master/jobs-data/linkedin_jobs_sample_36rows.csv')
    print('The sample jobs dataset has been loaded successfully!\n')

    job_sample = job_sample[['description', 'job_id']]
    job_sample = job_sample[1:3]
    print('The sample dataset has been filtered successfully!\n')
    print('Head of the sample:\n', job_sample.head())

    output = se.extractor(job_sample, 'job_id', text_columns=['description'])
    print('(LEGACY) The skills have been extracted from jobs data successfully...\n')

    # Save the extracted skills to a CSV file
    print(output)
    file_name = f'extracted_skills_for_{len(job_sample)}Jobs.csv'
    output.to_csv(file_name, index=False)
    print('(LEGACY) The extracted skills have been saved to the file named:', file_name)

    # Skill extraction from syllabi data
    print('\n\nLoading a sample dataset of 50 syllabi...')
    syllabi_sample = pd.read_csv('https://raw.githubusercontent.com/LAiSER-Software/datasets/refs/heads/master/syllabi-data/preprocessed_50_opensyllabus_syllabi_data.csv')
    print('The sample syllabi dataset has been loaded successfully!\n')

    syllabi_sample = syllabi_sample[['id', 'description', 'learning_outcomes']]
    syllabi_sample = syllabi_sample[1:3]
    print('The sample dataset has been filtered successfully!\n')
    print('Head of the sample:\n', syllabi_sample.head())

    output = se.extractor(syllabi_sample, 'id', text_columns=['description', 'learning_outcomes'], input_type='syllabus')
    print('The skills have been extracted from syllabi data successfully...\n')

    # Save the extracted skills to a CSV file
    print(output)
    file_name = f'extracted_skills_for_{len(syllabi_sample)}Syllabi.csv'
    output.to_csv(file_name, index=False)
    print('The extracted skills have been saved to the file named:', file_name)

def test_skill_extraction():
    """Test the refactored skill extraction functionality"""
    print("Testing refactored skill extraction...")

    try:

        # Test with transformer model (fallback case)
        print("\n1. Testing with GEMINI API...")
        extractor = SkillExtractorRefactored(
            model_id="gemini",
            api_key="<YOUR_GEMINI_API_KEY>",
            use_gpu=False
        )

        # Load job sample data
        job_sample = pd.read_csv('https://raw.githubusercontent.com/LAiSER-Software/datasets/refs/heads/master/jobs-data/linkedin_jobs_sample_36rows.csv')
        sample_data = job_sample[['description', 'job_id']].head(3)

        # Test basic extraction with single row
        print("\n2. Testing basic skill extraction with single row...")
        try:
            single_row_data = sample_data.iloc[0]  # Get first row as Series
            basic_skills = extractor.extract_skills(single_row_data, method="basic", input_type='job_desc', id_column="job_id")
            print(f"   Basic skills result: {basic_skills}")
            print(f"   Type: {type(basic_skills)}")

            # Save basic skills to CSV
            if basic_skills:
                basic_df = pd.DataFrame({
                    'job_id': [single_row_data['job_id']] * len(basic_skills),
                    'skill': basic_skills
                })
                basic_file_name = 'refactored_basic_skills_single_job.csv'
                basic_df.to_csv(basic_file_name, index=False)
                print(f"   (REFACTORED) Basic skills saved to: {basic_file_name}")
            else:
                print("   No basic skills to save")

        except Exception as e:
            print(f"   Basic extraction failed: {e}")

        # Test KSA extraction with single row
        print("\n3. Testing KSA skill extraction with single row...")
        try:
            single_row_data = sample_data.iloc[0]  # Get first row as Series
            ksa_skills = extractor.extract_skills(single_row_data, method="ksa", input_type='job_desc', id_column="job_id")
            print(f"   KSA skills result: {ksa_skills}")
            print(f"   Type: {type(ksa_skills)}")
            if isinstance(ksa_skills, list):
                print(f"   Number of skills: {len(ksa_skills)}")

                # Save KSA skills to CSV
                if ksa_skills:
                    ksa_df = pd.DataFrame(ksa_skills)
                    ksa_file_name = 'refactored_ksa_skills_single_job.csv'
                    ksa_df.to_csv(ksa_file_name, index=False)
                    print(f"   (REFACTORED) KSA skills saved to: {ksa_file_name}")
                else:
                    print("   No KSA skills to save")

        except Exception as e:
            print(f"   KSA extraction failed: {e}")

        # Test extract_and_align for multiple rows (correct method for DataFrames)
        print("\n4. Testing extract_and_align for multiple rows...")
        try:
            aligned_skills = extractor.extract_and_align(
                sample_data,
                id_column="job_id",
                text_columns=["description"],
                input_type='job_desc'
            )
            print(f"   Aligned skills result shape: {aligned_skills.shape}")
            print(f"   Aligned skills columns: {aligned_skills.columns.tolist()}")
            print(f"   Type: {type(aligned_skills)}")

            # Save the aligned skills to CSV file
            if not aligned_skills.empty:
                aligned_file_name = f'refactored_aligned_skills_for_{len(sample_data)}Jobs.csv'
                aligned_skills.to_csv(aligned_file_name, index=False)
                print(f"   (REFACTORED) Aligned skills saved to: {aligned_file_name}")
            else:
                print("   No aligned skills to save")

        except Exception as e:
            print(f"   Extract and align failed: {e}")

        # Test with pandas Series (your original use case)
        print("\n5. Testing with pandas Series...")
        try:
            series_data = sample_data.iloc[0]  # Get single row as Series
            print(f"   Series data type: {type(series_data)}")
            print(f"   Series keys: {series_data.index.tolist()}")

            series_skills = extractor.extract_skills(series_data, method="ksa", input_type='job_desc', id_column="job_id")
            print(f"   Series KSA skills result: {series_skills}")
            print(f"   Type: {type(series_skills)}")
        except Exception as e:
            print(f"   Series extraction failed: {e}")

        # Test with Gemini API (if API key available)
        print("\n6. Testing with Gemini API...")
        try:
            gemini_extractor = SkillExtractorRefactored(
                model_id="gemini",
                api_key="<YOUR_GEMINI_API_KEY>",
                use_gpu=False
            )
            print("   Gemini extractor initialized")
        except Exception as e:
            print(f"   Gemini initialization: {e}")

        # Test comprehensive extraction and save like legacy pipeline
        print("\n7. Testing comprehensive extraction (like legacy pipeline)...")
        try:
            # Process all sample data with extract_and_align
            comprehensive_skills = extractor.extract_and_align(
                sample_data,
                id_column="job_id",
                text_columns=["description"],
                input_type='job_desc'
            )

            print(f"   Comprehensive extraction completed")
            print(f"   Shape: {comprehensive_skills.shape}")

            # Save comprehensive results to CSV (similar to legacy format)
            if not comprehensive_skills.empty:
                comprehensive_file_name = f'refactored_extracted_skills_for_{len(sample_data)}Jobs.csv'
                comprehensive_skills.to_csv(comprehensive_file_name, index=False)
                print(f"   (REFACTORED) The extracted skills have been saved to: {comprehensive_file_name}")
                print(f"   Columns in output: {comprehensive_skills.columns.tolist()}")
            else:
                print("   No comprehensive skills extracted")

        except Exception as e:
            print(f"   Comprehensive extraction failed: {e}")

        print("\nAll tests completed!")

    except Exception as e:
        print(f"\n❌ Test failed with error: {e}")
        import traceback
        traceback.print_exc()
        return False

    return True

if __name__ == "__main__":
    legacy_check = test_legacy_skill_extraction()
    print("Legacy skill extraction test :", legacy_check)
    success = test_skill_extraction()
    sys.exit(0 if success else 1)


INFO 08-13 15:36:17 [__init__.py:235] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/laiser/llm_models/__init__.py:16: UserWarning: Some LLM model dependencies are not available: cannot import name 'GeminiAPI' from 'laiser.llm_models.gemini' (/usr/local/lib/python3.11/dist-packages/laiser/llm_models/gemini.py)
  warnings.warn(f"Some LLM model dependencies are not available: {e}")


Testing legacy skill extraction...


Initializing the Skill Extractor...
Loading ESCO skill taxonomy data...
Loading FAISS index for ESCO skills...
FAISS index for ESCO skills loaded successfully.
Found 'en_core_web_lg' model. Loading...
GPU is available. Using GPU for Large Language model initialization...


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

WARNING 08-13 15:37:06 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-13 15:37:06 [config.py:1604] Using max model len 32768
WARNING 08-13 15:37:08 [config.py:1084] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 08-13 15:37:08 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 08-13 15:37:08 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='TheBloke/Mistral-7B-Instruct-v0.1-AWQ', speculative_config=None, tokenizer='TheBloke/Mistral-7B-Instruct-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto,  device_co

...

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[get_ksa_details] No JSON match found in response for skill 'gather data'
The skills have been extracted from syllabi data successfully...

      Research ID                                        Description  \
0    661424964946  Course Description: you will be provided with ...   
1    661424964946  Course Description: you will be provided with ...   
2    661424964946  Course Description: you will be provided with ...   
3    661424964946  Course Description: you will be provided with ...   
4    661424964946  Course Description: you will be provided with ...   
5    661424964946  Course Description: you will be provided with ...   
6    661424964946  Course Description: you will be provided with ...   
7    661424964946  Course Description: you will be provided with ...   
8    661424964946  Course Description: you will be provided with ...   
9    661424964946  Course Description: you will be provided with ...   
10   661424964946  Course Description: you will be provided with ...

SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
